In [33]:
# import sys
import argparse
import numpy as np
import os
import cv2
import torch
import matplotlib.pyplot as plt
# from skimage.metrics import structural_similarity as ssim
# from skimage.measure import ransac
# from skimage.transform import FundamentalMatrixTransform, AffineTransform

from datetime import datetime

import torch
# from torchvision import transforms
# import torch.nn.functional as F
# from torch.utils import data
# from torchsummary import summary
# from pytorch_model_summary import summary

torch.manual_seed(9793047918980052389)
print('Seed:', torch.seed())

from utils.utils0 import *
from utils.utils1 import *
from utils.utils1 import ModelParams, print_summary
from utils import test

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

# Stub to warn about opencv version.
if int(cv2.__version__[0]) < 3: # pragma: no cover
  print('Warning: OpenCV 3 is not installed')

image_size = 256

Seed: 7764587458703181654
Device: cuda


In [34]:

# from utils.SuperPoint import SuperPointFrontend
# from utils.utils1 import transform_points_DVF
def test(model_name, models, model_params, timestamp):
    # model_name: name of the model
    # model: model to be tested
    # model_params: model parameters
    # timestamp: timestamp of the model
    print('Test function input:', model_name, models, model_params, timestamp)

    test_dataset = datagen(model_params.dataset, False, model_params.sup)

    model = [0]*len(models)
    # load the models one-by-one
    for i in range(len(models)):
        # if model is a string, load the model
        # if model is a loaded model, use the model
        if isinstance(models[i], str):
            print(f"\nLoading model: {models[i]}")
            model[i] = model_loader(model_name, model_params)
            buffer = io.BytesIO()
            torch.save(model[i].state_dict(), buffer)
            buffer.seek(0)
            model[i].load_state_dict(torch.load(models[i]))
            # print(f'Loaded model from {model[i]}')
        elif isinstance(models[i], nn.Module):
            print(f'Using model {model_name}')
            model[i] = models[i]

        # Set model to training mode
        model[i].eval()

    # Create output directory
    output_dir = f"output/{model_name}_{model_params.get_model_code()}_{timestamp}_ensemble_test"
    os.makedirs(output_dir, exist_ok=True)

    # Validate model
    # validation_loss = 0.0

    metrics = []
    # create a csv file to store the metrics
    csv_file = f"{output_dir}/metrics.csv"

    with torch.no_grad():
        testbar = tqdm(test_dataset, desc=f'Testing:')
        for i, data in enumerate(testbar, 0):
            # Get images and affine parameters
            source_image, target_image, affine_params_true, points1, points2, points1_2_true = data

            source_image = source_image.requires_grad_(True).to(device)
            target_image = target_image.requires_grad_(True).to(device)
            # add gradient to the matches
            points1 = points1.requires_grad_(True).to(device)
            points2 = points2.requires_grad_(True).to(device)

            # TODO: how to repeat the test?
            # 1. until the affine parameters are not change anymore
            # 2. until the mse is not change anymore
            # 3. until the mse is not change anymore and the affine parameters are not change anymore

            # use for loop with a large number of iterations 
            # check TRE of points1 and points2
            # if TRE grows larger than the last iteration, stop the loop
            TRE_last = np.inf
            MSE_last = np.inf
            mse12 = 0
            tre12 = 0

            mse_before_first, tre_before_first, mse12_image_before_first, \
                ssim12_image_before_first = 0, 0, 0, 0
            mse_before, tre_before, mse12_image, ssim12_image = 0, 0, 0, 0

            rep = 2
            votes = [np.inf] * rep  # Initialize a list to store the votes for each model
            mse_list = [np.inf] * 5
            tre_list = [np.inf] * 5
            no_improve = 0

            for j in range(rep):
                for k in range(len(models)):
                    # Forward + backward + optimize
                    outputs = model[k](source_image, target_image, points1)
                    transformed_source_affine = outputs[0]
                    affine_params_predicted = outputs[1]
                    points1_2_predicted = outputs[2]

                    # if i is an odd number
                    if i % 2 == 1 and i < 10 and model_params.plot == 0:
                        plot_ = True
                    else:
                        plot_ = False

                    results = DL_affine_plot(f"test_{i}", output_dir,
                        f"{i+1}", f"rep{j:02d}_{k}", source_image[0, 0, :, :].cpu().numpy(), 
                        target_image[0, 0, :, :].cpu().numpy(), 
                        transformed_source_affine[0, 0, :, :].cpu().numpy(),
                        points1[0].cpu().detach().numpy().T, 
                        points2[0].cpu().detach().numpy().T, 
                        points1_2_predicted[0].cpu().detach().numpy().T, None, None, 
                        affine_params_true=affine_params_true,
                        affine_params_predict=affine_params_predicted, 
                        heatmap1=None, heatmap2=None, plot=plot_)

                    mse_before = results[1]
                    tre_before = results[3]
                    mse12_image_before = results[5]
                    ssim12_image_before = results[7]

                    mse12 = results[2]
                    tre12 = results[4]
                    mse12_image = results[6]
                    ssim12_image = results[8]

                    mse_list[k] = mse12
                    tre_list[k] = tre12

                    if j == 0 and k == 0:
                        mse_before_first, tre_before_first, mse12_image_before_first, \
                            ssim12_image_before_first = mse_before, tre_before, mse12_image_before, ssim12_image_before
                        # print(mse_before_first, tre_before_first, mse12_image_before_first, ssim12_image_before_first)
                
                # print(f"Pair {i}, Rep {j}: {mse_list}, {tre_list}")
                # the lowset mse12 and tre12 and its index
                mse12, tre12 = np.min(mse_list), np.min(tre_list)
                best_mse = np.argmin([mse_list])  # Find the index of the model with the best results
                best_tre = np.argmin([tre_list])  # Find the index of the model with the best results

                # print(f"Pair {i}, Rep {j}: {mse12}, {tre12}, best model: {best_mse}, {best_tre}")
                
                # if any element in tre_list is nan, use the model with the lowest mse
                if np.isnan(tre12):
                    votes[j] = best_tre
                else:
                    votes[j] = best_mse
                    best_tre = best_mse
                    tre12 = mse12
                    TRE_last = MSE_last

                # print(f"Pair {i}, Rep {j}: {mse12}, {tre12}, best model: {best_tre} {best_mse}")

                outputs = model[best_tre](source_image, target_image, points1)
                transformed_source_affine = outputs[0]
                affine_params_predicted = outputs[1]
                points1_2_predicted = outputs[2]
                
                if model_params.plot == 1 and i < 50:
                    plot_ = True
                else:
                    plot_ = False

                results = DL_affine_plot(f"test_{i}", output_dir,
                    f"{i+1}", f"rep{j:02d}_{best_tre}", source_image[0, 0, :, :].cpu().numpy(),
                    target_image[0, 0, :, :].cpu().numpy(),
                    transformed_source_affine[0, 0, :, :].cpu().numpy(),
                    points1[0].cpu().detach().numpy().T,
                    points2[0].cpu().detach().numpy().T,
                    points1_2_predicted[0].cpu().detach().numpy().T, None, None,
                    affine_params_true=affine_params_true,
                    affine_params_predict=affine_params_predicted,
                    heatmap1=None, heatmap2=None, plot=plot_)
                    
                # mse_before = results[1]
                # tre_before = results[3]
                # mse12_image_before = results[5]
                # ssim12_image_before = results[7]

                mse12 = results[2]
                tre12 = results[4]
                mse12_image = results[6]
                ssim12_image = results[8]

                points1 = points1_2_predicted.clone()
                source_image = transformed_source_affine.clone()

                # apply the best model to this pair
                # if tre12 < TRE_last and mse12 < MSE_last:
                if tre12 < TRE_last:
                    TRE_last = tre12
                    MSE_last = mse12
                    no_improve -= 1
                
                else:
                    # tre12 = TRE_last
                    # mse12 = MSE_last
                    no_improve += 1

                # if there is no improvement for 2 reps, stop the iteration
                if no_improve > 2:
                    break

            # print(f'\nEnd register pair {i}')
            # print(f'Votes: {votes}\n')
            # break

            # append metrics to metrics list
            new_entry = [i, mse_before_first, mse12, tre_before_first, tre12, mse12_image_before_first, mse12_image, \
                            ssim12_image_before_first, ssim12_image, np.max(points1_2_predicted.shape), votes]
            metrics.append(new_entry)
            # print(f"Pair {i}: {new_entry}")
            # break

    metrics_ = []
    with open(csv_file, 'w', newline='') as file:

        writer = csv.writer(file)
        writer.writerow(["index", "mse_before", "mse12", "tre_before", "tre12", "mse12_image_before", "mse12_image", 
                         "ssim12_image_before", "ssim12_image", "num_points", "votes"])
        for i in range(len(metrics)):
            writer.writerow(metrics[i])

        # print(metrics[40:45])
        
        # drop the last column of the array 'metrics'
        metrics = [metrics[i][0:-1] for i in range(len(metrics))]
        metrics_ = metrics.copy()
        metrics = np.array(metrics)

        # print(metrics[40:45])

        # metrics = metrics[:, :8]
        nan_mask = np.isnan(metrics).any(axis=1)
        metrics = metrics[~nan_mask]

        print(metrics[40])

        avg = ["average", np.mean(metrics[:, 1]), np.mean(metrics[:, 2]), np.mean(metrics[:, 3]), np.mean(metrics[:, 4]), 
            np.mean(metrics[:, 5]), np.mean(metrics[:, 6]), np.mean(metrics[:, 7]), np.mean(metrics[:, 8])]
        std = ["std", np.std(metrics[:, 1]), np.std(metrics[:, 2]), np.std(metrics[:, 3]), np.std(metrics[:, 4]), 
            np.std(metrics[:, 5]), np.std(metrics[:, 6]), np.std(metrics[:, 7]), np.std(metrics[:, 8])]
        # avg = ["average", np.mean(metrics[:, 1:9], axis=0)]
        # std = ["std", np.std(metrics[:, 1:9], axis=0)]
        writer.writerow(avg)
        writer.writerow(std)

    print(f"The test results are saved in {csv_file}")

    return csv_file, metrics_


In [35]:

# Access the values of the command-line arguments
dataset = 0
sup = 1
image = 1
heatmaps = 0
loss_image = 0
num_epochs = 1
learning_rate = 1e-3
decay_rate = 0.96
model = 'DHR'
model_path = None
plot = 2

# model_path = 'trained_models/' + args.model_path
model_path = ['DHR_11100_0.001_0_5_100_20240509-155916.pth', 'DHR_21100_0.001_0_5_100_20240509-160207.pth',
            'DHR_31100_0.001_0_10_100_20240508-120807.pth', 'DHR_41100_0.001_0_5_100_20240509-133824.pth',
            'DHR_51100_0.001_0_5_100_20240509-140837.pth']
# add 'trained_models/' in front of each element of model_path
model_path = ['trained_models/' + path for path in model_path]

model_params = ModelParams(dataset=dataset, sup=sup, image=image, loss_image=loss_image, 
                            num_epochs=num_epochs, learning_rate=learning_rate, 
                            decay_rate=decay_rate, 
                            plot=plot)
model_params.print_explanation()

timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")

Model name:  dataset0_sup0_image1_points0_loss_image0
Model code:  00100_0.001_0_1_1
Model params:  {'dataset': 0, 'sup': 0, 'image': 1, 'points': 0, 'loss_image_case': 0, 'loss_image': MSELoss(), 'loss_affine': <utils.utils1.loss_affine object at 0x7fad80bb7c10>, 'learning_rate': 0.001, 'decay_rate': 0.96, 'start_epoch': 0, 'num_epochs': 1, 'batch_size': 1, 'model_name': 'dataset0_sup0_image1_points0_loss_image0'}

Model name:  dataset0_sup0_image1_points0_loss_image0
Model code:  00100_0.001_0_1_1
Dataset used:  Actual eye
Supervised or unsupervised model:  Unsupervised
Loss image type:  Image used
Points used:  Points not used
Loss function case:  0
Loss function for image:  MSELoss()
Loss function for affine:  <utils.utils1.loss_affine object at 0x7fad80bb7c10>
Learning rate:  0.001
Decay rate:  0.96
Start epoch:  0
Number of epochs:  1
Batch size:  1




In [36]:
print(f"\nTesting the trained model: {model} +++++++++++++++++++++++")

csv_file, metrics = test(model, model_path, model_params, timestamp)
print("Test model finished +++++++++++++++++++++++++++++")


Testing the trained model: DHR +++++++++++++++++++++++
Test function input: DHR ['trained_models/DHR_11100_0.001_0_5_100_20240509-155916.pth', 'trained_models/DHR_21100_0.001_0_5_100_20240509-160207.pth', 'trained_models/DHR_31100_0.001_0_10_100_20240508-120807.pth', 'trained_models/DHR_41100_0.001_0_5_100_20240509-133824.pth', 'trained_models/DHR_51100_0.001_0_5_100_20240509-140837.pth'] dataset0_sup0_image1_points0_loss_image0 20240512-103540
Test eye dataset
Number of testing data:  100

Loading model: trained_models/DHR_11100_0.001_0_5_100_20240509-155916.pth
Using DHR difference

Loading model: trained_models/DHR_21100_0.001_0_5_100_20240509-160207.pth
Using DHR difference

Loading model: trained_models/DHR_31100_0.001_0_10_100_20240508-120807.pth
Using DHR difference

Loading model: trained_models/DHR_41100_0.001_0_5_100_20240509-133824.pth
Using DHR difference

Loading model: trained_models/DHR_51100_0.001_0_5_100_20240509-140837.pth
Using DHR difference


Testing::  41%|████      | 41/100 [00:09<00:13,  4.33it/s]/home/pkhamchuai/codes/spppt-2/.venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/pkhamchuai/codes/spppt-2/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
Testing:: 100%|██████████| 100/100 [00:24<00:00,  4.11it/s]

[4.00000000e+01 4.08337500e+03 3.72340771e+03 9.03580093e+01
 8.62562790e+01 3.98883931e-02 2.69634128e-02 4.10601044e-01
 4.69665303e-01 8.00000000e+00]
The test results are saved in output/DHR_00100_0.001_0_1_1_20240512-103540_ensemble_test/metrics.csv
Test model finished +++++++++++++++++++++++++++++


In [37]:
for i in metrics:
    print(i)

[0, 952.5, 789.3545, 43.490982, 39.590244, 0.09467417, 0.093663536, 0.3826547475335451, 0.4048030437547383, 2]
[1, 438.91428, 221.28577, 28.807297, 20.44842, 0.02356157, 0.016647225, 0.4219869167820732, 0.45322953011712797, 35]
[2, 1.0068493, 1.889098, 1.2436951, 1.7349043, 0.0011295322, 0.0011180014, 0.8906353716834011, 0.8846335494828471, 146]
[3, 162.27272, 86.254974, 17.178314, 11.773314, 0.012607125, 0.009299504, 0.6429356175700988, 0.6883948700764612, 88]
[4, 193.71666, 173.0996, 18.64133, 17.817951, 0.027227499, 0.026457302, 0.3895056947570645, 0.4200614406237855, 30]
[5, 127.05556, 83.55985, 11.901311, 8.858511, 0.010623909, 0.007911285, 0.5504893353454324, 0.6200276921349168, 72]
[6, 140.25, 141.977, 16.085976, 16.40456, 0.017356073, 0.018795567, 0.45740179260941316, 0.4550833199368048, 18]
[7, 392.42856, 341.30896, 27.623726, 25.68054, 0.04754443, 0.0446832, 0.23586346569837047, 0.25737604794764907, 35]
[8, 3784.6667, 3585.2021, 86.056435, 84.210464, 0.092122495, 0.09094159, 

In [38]:
metrics = np.array(metrics)
nan_mask = np.isnan(metrics).any(axis=1)
metrics = metrics[~nan_mask]
np.mean(metrics, axis=0)

array([4.95858586e+01, 5.04985432e+02, 4.25540972e+02, 2.58785015e+01,
       2.30344701e+01, 3.15154436e-02, 2.82837305e-02, 4.50214391e-01,
       4.83310901e-01, 4.45252525e+01])

In [39]:
metrics.shape

(99, 10)

In [40]:
metrics[41]

array([4.20000000e+01, 4.08125000e+02, 2.13804993e+02, 2.82771091e+01,
       1.94525051e+01, 3.21540907e-02, 2.42713634e-02, 5.18360528e-01,
       5.50406915e-01, 3.60000000e+01])